In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 10.5 MB/s eta 0:00:00


In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
import torch
from IPython.display import Image, clear_output  # to display images

from os import listdir
from os.path import isfile, join
import yaml

In [ ]:
import os
def count_file(folder_path):
    # Get the list of files in the folder
    file_list = os.listdir(folder_path)

    # Count the number of files
    num_files = len(file_list)

    # Print the number of files
    print(f"Folder '{folder_path}' : {num_files} files.")

# **Root** ( change this for each process)

ĐỔI path_of_root thành path của nơi để folder gốc

In [ ]:
# Nơi để folder
path_of_root="/content/drive/MyDrive/"

#data png unzip
vinbigdata="/content/Process 1/"

#Working for Normal
working = path_of_root + "DS312 - Xử lí ảnh y khoa/Project/Working/Process1_V8/"

In [ ]:
path_of_data=working + 'data_yaml/vinbigdata.yaml'

# Unzip Data

In [ ]:
%cd /content

/content


In [ ]:
zip = path_of_root + "DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/Process 1.zip"
!unzip "{zip}"

In [ ]:
count_file("/content/Process 1")

# Import Data

train.csv

In [ ]:
train_df = pd.read_csv(path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/train.csv')
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353


In [ ]:
len(train_df['image_id'].unique())

3515

val.csv

In [ ]:
val_df = pd.read_csv(path_of_root +'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/val.csv')
len(val_df['image_id'].unique())

439

test.csv

In [ ]:
test_df = pd.read_csv(path_of_root +'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/test.csv')
len(test_df['image_id'].unique())

440

read image file

In [ ]:
train_df['image_path'] = vinbigdata+train_df.image_id+'.png'
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336,/content/Process 1/9a5094b2563a1ef3ff50dc5c7ff...
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880,/content/Process 1/051132a778e61a86eb147c7c6f5...
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072,/content/Process 1/1c32170b4af4ce1a3030eb81677...
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555,/content/Process 1/0c7a38f293d5f5e4846aa4ca6db...
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353,/content/Process 1/47ed17dcb2cbeec15182ed335a8...


In [ ]:
val_df['image_path'] = vinbigdata+val_df.image_id+'.png'
val_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,d3637a1935a905b3c326af31389cb846,Aortic enlargement,0,R10,1329.0,743.0,1521.0,958.0,2304,2880,/content/Process 1/d3637a1935a905b3c326af31389...
1,0a14aeaf02d42990d8bb5d55270b7274,ILD,5,R9,593.0,1597.0,1111.0,2207.0,2774,3043,/content/Process 1/0a14aeaf02d42990d8bb5d55270...
2,ebf4ffbcf6de699bf9deab70f0dee8ff,Cardiomegaly,3,R9,876.0,1810.0,2336.0,2610.0,2779,2912,/content/Process 1/ebf4ffbcf6de699bf9deab70f0d...
3,eb6c714df22142229464c6b83e47d7d6,Infiltration,6,R8,439.0,1810.0,961.0,2452.0,2532,3073,/content/Process 1/eb6c714df22142229464c6b83e4...
4,fa4c38b3c5e53e1e84bd0923f2d8e480,Aortic enlargement,0,R9,1397.0,849.0,1661.0,1200.0,2304,2880,/content/Process 1/fa4c38b3c5e53e1e84bd0923f2d...


In [ ]:
test_df['image_path'] = vinbigdata+test_df.image_id+'.png'
test_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,18a61a07e6f5f13ebfee57fa36cd8b6f,Pulmonary fibrosis,13,R9,393.0,283.0,822.0,643.0,2304,2880,/content/Process 1/18a61a07e6f5f13ebfee57fa36c...
1,d7aea6f37597df566725da73655edcc2,Infiltration,6,R8,1768.0,1137.0,2346.0,2020.0,2594,3395,/content/Process 1/d7aea6f37597df566725da73655...
2,0cfae3f740f87ca3b96dec765d06fad0,Aortic enlargement,0,R9,1142.0,738.0,1451.0,1119.0,2304,2880,/content/Process 1/0cfae3f740f87ca3b96dec765d0...
3,b42e930c16c0166dbeae813b47bb8b07,Pleural thickening,11,R9,1972.0,401.0,2250.0,574.0,3072,3072,/content/Process 1/b42e930c16c0166dbeae813b47b...
4,d07557904cbe57fc1bdebac1e8aeefa1,Nodule/Mass,8,R9,378.0,899.0,403.0,929.0,2304,2880,/content/Process 1/d07557904cbe57fc1bdebac1e8a...


In [ ]:
classes = train_df['class_name'].unique().tolist()
classes =sorted(classes)
classes

# Prepare DATA train, val, test

In [ ]:
train_files = train_df['image_path'].unique()
val_files   = val_df['image_path'].unique()
test_files  = test_df['image_path'].unique()
len(train_files), len(val_files), len(test_files)

(3515, 439, 440)

## Copying Files
- ĐÃ CÓ DATA TRONG FOLDER THÌ KHÔNG CẦN CHẠY

In [ ]:
count_file(path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/vinbigdata_label')

In [ ]:
os.makedirs(working+'vinbigdata/labels/train', exist_ok = True)
os.makedirs(working+'vinbigdata/labels/val', exist_ok = True)
os.makedirs(working+'vinbigdata/labels/test', exist_ok = True)


os.makedirs(working+'vinbigdata/images/train', exist_ok = True)
os.makedirs(working+'vinbigdata/images/val', exist_ok = True)
os.makedirs(working+'vinbigdata/images/test', exist_ok = True)

label_dir = path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/vinbigdata_label'

In [ ]:
for file in tqdm(train_files):
    shutil.copy(file, working+'vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/train')

for file in tqdm(val_files):
    shutil.copy(file, working+'vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/val')

for file in tqdm(test_files):
    shutil.copy(file, working+'vinbigdata/images/test')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/test')

# YOLO v8 make .ymal and folder data
- ĐÃ CÓ DATA TRONG FOLDER THÌ KHÔNG CẦN CHẠY

In [ ]:
os.makedirs(working+'data_yaml', exist_ok = True)

with open(join( working , 'data_yaml/train.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/train/*'):
        f.write(path+'\n')

with open(join( working , 'data_yaml/val.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/val/*'):
        f.write(path+'\n')

with open(join( working , 'data_yaml/test.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/test/*'):
        f.write(path+'\n')


data = dict(
    test   =  join( working , 'data_yaml/test.txt' ),
    val   =  join( working , 'data_yaml/val.txt' ),
    train =  join( working , 'data_yaml/train.txt') ,
    nc    = 14,
    names = classes
    )

with open(join( working , 'data_yaml/vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( working , 'data_yaml/vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())

# Training
- BỎ QUA NẾU ĐÃ TRAIN

CHECK FILE BEFORE TRAIN:
- Train : 3515
- Val : 439
- Test : 440

if not enough files : reconnect the Drive until enough

In [ ]:
count_file(working+'vinbigdata/images/train'), count_file(working+'vinbigdata/labels/train')

In [ ]:
count_file(working+'vinbigdata/images/val'), count_file(working+'vinbigdata/labels/val')

In [ ]:
count_file(working+'vinbigdata/images/test'), count_file(working+'vinbigdata/labels/test')

TRAIN

In [ ]:
os.makedirs(working+'YOLO', exist_ok = True)

os.chdir(working+'YOLO')

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
count_file(working+'vinbigdata/images/train'), count_file(working+'vinbigdata/labels/train')

In [ ]:
!pwd

In [ ]:
path_of_model_yolov8 = path_of_root + "DS312 - Xử lí ảnh y khoa/Project/Working/yolov8/yolov8_pt/yolov8l/yolov8l.pt"

In [ ]:
!yolo task=detect mode=train model="$path_of_model_yolov8" data="$path_of_data" epochs= 20 imgsz =640

# path runs of train, predict
- Sau khi training thì sẽ lưu vào path "runs/detect/.."
- Chọn train_path sau khi train ( vd : train_path='train2' )

In [ ]:
import os

folder_path = working+ "YOLO/runs/detect/"
folders = os.listdir(folder_path)

train_folders = [folder for folder in folders if folder.startswith("train")]
train_folders.sort(reverse=True)  # Sắp xếp giảm dần theo thứ tự từ điển

if len(train_folders) > 0:
    latest_train_folder = train_folders[0]
    print(latest_train_folder)
else:
    print("Không tìm thấy folder bắt đầu bằng 'train'.")

train2


In [ ]:
train_path=latest_train_folder

# EVALUATE

In [ ]:
path_of_model_train = working+ f"YOLO/runs/detect/{train_path}/weights/best.pt"

In [ ]:
os.chdir(working+'YOLO')

## Valid


In [ ]:
!yolo task=detect mode=val model="$path_of_model_train" data="$path_of_data"

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43617402 parameters, 0 gradients
100% 755k/755k [00:00<00:00, 17.1MB/s]
val: Scanning /content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Process1_V8/vinbigdata/labels/val.cache... 439 images, 0 backgrounds, 0 corrupt: 100% 439/439 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 28/28 [01:09<00:00,  2.48s/it]
                   all        439       3511      0.416      0.331      0.314      0.162
    Aortic enlargement        439        748      0.714      0.608      0.705      0.426
           Atelectasis        439         22      0.441      0.273      0.228      0.119
         Calcification        439         88      0.181      0.148     0.0878     0.0376
          Cardiomegaly        439        565      0.747      0.638      0.704      0.472
         Consolidation        439        

## Test

In [ ]:
!yolo task=detect mode=val model="$path_of_model_train" data="$path_of_data" split=test

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43617402 parameters, 0 gradients
val: Scanning /content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Process1_V8/vinbigdata/labels/test... 440 images, 0 backgrounds, 0 corrupt: 100% 440/440 [03:29<00:00,  2.10it/s]
val: New cache created: /content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Process1_V8/vinbigdata/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 28/28 [00:24<00:00,  1.15it/s]
                   all        440       3528      0.374      0.301      0.291      0.148
    Aortic enlargement        440        757      0.679      0.605      0.697       0.43
           Atelectasis        440         24      0.225      0.125      0.171     0.0939
         Calcification        440        131       0.22      0.107     0.0876     0.0355
          Cardiomegaly        440